<a href="https://colab.research.google.com/github/vitroid/GenIce-core/blob/main/example-bjerrum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/genice-dev/GenIce-core/blob/main/example-bjerrum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## for Google Colaboratory


In [ ]:
try:
    import google.colab
    %pip install git+https://github.com/genice-dev/GenIce-core.git py3dmol
except:
    pass

In [ ]:
import networkx as nx
import numpy as np
import plotly.graph_objects as go


# a useful function to draw a graph with plotly.
def draw_graph(g: nx.Graph, pos: dict, fixed: list = None, dopant: list = []):
    # draw the graph with edges and labels in 3D
    pos = np.array([pos[i] for i in sorted(g.nodes())])

    # 通常の辺のリスト
    normal_edges = [
        (i, j)
        for i, j in g.edges()
        if fixed is None or [i, j] not in fixed and [j, i] not in fixed
    ]
    # 固定された辺のリスト
    fixed_edges = fixed if fixed is not None else []

    normal_nodes = [i for i in g.nodes() if i not in dopant]
    dopant_nodes = [i for i in dopant]

    normal_pos = pos[normal_nodes]
    dopant_pos = pos[dopant_nodes]

    fig = go.Figure(
        data=[
            # ノードの表示
            go.Scatter3d(
                x=normal_pos[:, 0],
                y=normal_pos[:, 1],
                z=normal_pos[:, 2],
                mode="markers+text",
                marker=dict(size=2, color="blue"),
                text=[str(i) for i in normal_nodes],
                textposition="top center",
            ),
            go.Scatter3d(
                x=dopant_pos[:, 0],
                y=dopant_pos[:, 1],
                z=dopant_pos[:, 2],
                mode="markers+text",
                marker=dict(size=4, color="red"),
                text=[str(i) for i in dopant_nodes],
                textposition="top center",
            ),
            # 通常の辺の表示
            *[
                go.Scatter3d(
                    x=[pos[i, 0], pos[j, 0]],
                    y=[pos[i, 1], pos[j, 1]],
                    z=[pos[i, 2], pos[j, 2]],
                    mode="lines",
                    line=dict(color="gray", width=2),
                    hoverinfo="none",
                )
                for i, j in normal_edges
            ],
            # 固定された辺（矢印）の表示
            *[
                go.Scatter3d(
                    x=[pos[i, 0], pos[j, 0]],
                    y=[pos[i, 1], pos[j, 1]],
                    z=[pos[i, 2], pos[j, 2]],
                    mode="lines",
                    line=dict(color="green", width=3),
                    hoverinfo="none",
                )
                for i, j in fixed_edges
            ],
            # 矢印の先端（コーン）の表示
            *[
                go.Cone(
                    x=[(pos[j, 0] - pos[i, 0]) * 0.8 + pos[i, 0]],
                    y=[(pos[j, 1] - pos[i, 1]) * 0.8 + pos[i, 1]],
                    z=[(pos[j, 2] - pos[i, 2]) * 0.8 + pos[i, 2]],
                    u=[(pos[j, 0] - pos[i, 0]) * 0.4],
                    v=[(pos[j, 1] - pos[i, 1]) * 0.4],
                    w=[(pos[j, 2] - pos[i, 2]) * 0.4],
                    colorscale=[[0, "green"], [1, "green"]],
                    showscale=False,
                )
                for i, j in fixed_edges
            ],
        ]
    )
    fig.show()

## 1. Prepare an undirected graph.


In [ ]:
from logging import getLogger, DEBUG

logger = getLogger()
logger.setLevel(DEBUG)

g = nx.dodecahedral_graph()  # dodecahedral 20mer and an additional node at the center
pos = nx.spring_layout(
    g,
    dim=3,
)

# add a new node at the center
g.add_node(20)
pos[20] = (0, 0, 0)
g.add_edge(0, 20)
g.add_edge(6, 20)
g.add_edge(13, 20)
g.add_edge(17, 20)
pos = nx.spring_layout(g, dim=3, pos=pos)

# average edge length
L = [np.linalg.norm(pos[a] - pos[b]) for a, b in g.edges()]

# scale the shape
pos = {k: v * 2.76 / np.mean(L) for k, v in pos.items()}

# pos = nx.spring_layout(g, pos=pos)

draw_graph(g, pos)

Fix edges on the 0th water molecule to make it an Eigen-type.


In [ ]:
import random

eigen = 0
neighbors = list(nx.neighbors(g, eigen))
assert len(neighbors) == 4

accept = random.choice(neighbors)  # one in
donate = set(neighbors) - set([accept])  # three outs

fixed = []
fixed.append([accept, eigen])
for d in donate:
    fixed.append([eigen, d])
# 3-out, 1-in

In the same way, fix edges on the 13th water molecule to make it hydroxy-like.


In [ ]:
oh = 13
neighbors = list(nx.neighbors(g, oh))
assert len(neighbors) == 4

donate = random.choice(neighbors)  # one out
accept = set(neighbors) - set([donate])  # three ins

fixed.append([oh, donate])
for a in accept:
    fixed.append([a, oh])
# 1-out, 3-in

draw_graph(g, pos, fixed=fixed)

And arrange all other edges appropreately.


In [ ]:
import genice_core

# set orientations of the hydrogen bonds.
dg = genice_core.ice_graph(g, vertexPositions=pos, fixedEdges=nx.DiGraph(fixed))

draw_graph(g, pos, fixed=dg.edges())

Put a virtual sites for the dangling bonds.


In [ ]:
def decide1(v1, v2, v3):
    v4 = -(v1 + v2 + v3)
    return v4 / np.linalg.norm(v4)


def decide2(v1, v2):
    d = v1 + v2
    d /= np.linalg.norm(d)
    v = np.cross(v1, v2)
    v /= np.linalg.norm(v)
    d *= -sqrt(1.0 / 3)
    v *= sqrt(2.0 / 3)
    return d + v, d - v


def add_virtual_edge(v, g, pos, dg, edge_length=3.0):
    new_node = len(pos)
    pos[new_node] = pos[O] + v * edge_length
    g.add_edge(O, new_node)
    if dg.in_degree(O) > dg.out_degree(O):
        dg.add_edge(O, new_node)
    else:
        dg.add_edge(new_node, O)


def add_virtual_sites(O, g, pos, dg, edge_length=3.0):
    if g.degree(O) == 3:
        neighbors = [node for node in g[O]]
        v1 = pos[neighbors[0]] - pos[O]
        v2 = pos[neighbors[1]] - pos[O]
        v3 = pos[neighbors[2]] - pos[O]
        v1 /= np.linalg.norm(v1)
        v2 /= np.linalg.norm(v2)
        v3 /= np.linalg.norm(v3)
        v4 = decide1(v1, v2, v3)
        add_virtual_edge(v4, g, pos, dg, edge_length=edge_length)
    elif g.degree(O) == 2:
        neighbors = [node for node in g[O]]
        v1 = pos[neighbors[0]] - pos[O]
        v2 = pos[neighbors[1]] - pos[O]
        v1 /= np.linalg.norm(v1)
        v2 /= np.linalg.norm(v2)
        v3, v4 = decide2(v1, v2)
        add_virtual_edge(v3, g, pos, dg, edge_length=edge_length)
        add_virtual_edge(v4, g, pos, dg, edge_length=edge_length)


g_decorated = g.copy()
dg_decorated = dg.copy()
pos_decorated = pos.copy()

for O in g:
    # if the oxygen has edges less than 4,
    if g.degree(O) < 4:
        add_virtual_sites(O, g_decorated, pos_decorated, dg_decorated, edge_length=2.76)

draw_graph(g_decorated, pos_decorated, fixed=dg_decorated.edges())

Put hydrogens along the directed edges.


In [ ]:
oxygens = [pos[node] for node in g]
hydrogens = [
    pos_decorated[s] + (pos_decorated[d] - pos_decorated[s]) * 1.0 / 2.76
    for s, d in dg_decorated.edges()
    if s < len(pos)  # do not put hydrogen on the edge starting from a virtual node
]

Display with py3dmol.


In [ ]:
import py3Dmol


def gen_gro(oxygens, hydrogens):
    gro = f"genice-core\n{len(oxygens)+len(hydrogens)}\n"
    for label, (x, y, z) in enumerate(oxygens):
        gro += f"{1:>5}O    O{label:<4}{label+1:>4}{x/10.0:>8.3f}{y/10.0:>8.3f}{z/10.0:>8.3f}\n"
    for label, (x, y, z) in enumerate(hydrogens):
        gro += f"{1:>5}H    H{label:<4}{label+1:>4}{x/10.0:>8.3f}{y/10.0:>8.3f}{z/10.0:>8.3f}\n"
    gro += "1 1 1\n"
    return gro


gro = gen_gro(oxygens, hydrogens)
with open("dodeca.gro", "w") as f:
    f.write(gro)

# show
view = py3Dmol.view()
view.addModel(gro, "gro")
view.setStyle(
    {
        "stick": {
            "radius": 0.15,
        },
        "sphere": {"radius": 0.35},
    }
)
view.addUnitCell()
view.zoomTo()
view.show()
# view.png()

To make the ionic defects to be Bjerrums, move a proton on H3O to OH.


In [ ]:
import random

L = random.choice([node for node in dg.successors(eigen)])
D = random.choice([node for node in dg.predecessors(oh)])

dg_decorated.remove_edge(eigen, L)
dg_decorated.add_edge(oh, D)

In [ ]:
hydrogens = [
    pos_decorated[s] + (pos_decorated[d] - pos_decorated[s]) * 1.0 / 2.76
    for s, d in dg_decorated.edges()
    if s < len(pos)  # do not put hydrogen on the edge starting from a virtual node
]

In [ ]:
import py3Dmol

gro = gen_gro(oxygens, hydrogens)
with open("dodeca.gro", "w") as f:
    f.write(gro)
# show
view = py3Dmol.view()
view.addModel(gro, "gro")
view.setStyle(
    {
        "stick": {
            "radius": 0.15,
        },
        "sphere": {"radius": 0.35},
    }
)
view.addUnitCell()
view.zoomTo()
view.show()